# Segregate CM data

###Specifications

####  Step 01:

Add CM monthly and Yearly data together but to differentiate data, add a separate column name "Plan Type" and label monthly data as "Monthly" and Yearly data as "Annual" 


#### Step 2:

Now we need to separate/sort out data by different platform data using "Charge ID". Create separate sheets/Tab to move below data 

o	Transactions starting with ch_& re_ or py_ -----> Stripe

o	Transactions Starting with number -------> Paypal  ( Or in the middle separate by -Com)

o	Transactions starting with tr_ -----> Apple & Google


### Script
---

#### Imports,prepartions and functions

In [1]:
import pandas as pd
import os
!pip install XlsxWriter
import xlsxwriter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 2.9 MB/s eta 0:00:00


###### Preparations
---


In [2]:
#Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')

#Set file paths
path = "/content/gdrive/My Drive/Data Science 2022/CM Audit/"

inpathCM = path + "CM Data/2023/2023.02/"

outpath = path + "Monthly Output/2023.02/"
filename = "CM_2023.02.xlsx"
#Create Results folder if missing
if not os.path.isdir(outpath):
  os.mkdir(outpath)


Mounted at /content/gdrive


#### Segregate ChartMogul

In [3]:
###### Combine Monthly and yearly data differentiating using 'Plan Type' column ############
for file in os.listdir(inpathCM ):
  print (file)
  
  #For csv format
  if file[-3:] == "csv":  
    if "Mon" in file:     #Monthly data
      dfmon = pd.read_csv(inpathCM + "/" +  file , na_filter=False, index_col=False)
    else:
      dfyear = pd.read_csv(inpathCM + "/" +  file , na_filter=False, index_col=False)
      #dfyear = pd.read_csv(inpathCM + item + "/" +  file ,encoding="latin")
  
  else:
    if "Mon" in file:     #Monthly data
      dfmon = pd.read_excel(inpathCM + "/" +  file , na_filter=False, index_col=False)
    else:
      dfyear = pd.read_excel(inpathCM + "/" +  file , na_filter=False,  index_col=False)
      #dfyear = pd.read_csv(inpathCM + item + "/" +  file ,encoding="latin")

colcnt = dfmon.shape[1]
dfmon.insert(colcnt,"Plan Type", "Monthly")
dfyear.insert(colcnt,"Plan Type", "Yearly" )

#Concat the datasets
dfcombine = pd.concat([dfmon,dfyear])
#dfcombine = dfcombine0[dfcombine0['Payment / Refund'] == "payment"]
###### Segregate based on 'Charge Id' ##########
#Transactions starting with ch_& re_ or py_ -----> Stripe
dfs = dfcombine[dfcombine['Charge ID'].str.contains("ch_|re_|py_") ]
#Transactions Starting with number -------> Paypal ( Or in the middle separate by -Com)
#dfpp = dfcombine[dfcombine['Charge ID'].str.contains("-Com") ]
dfpp = dfcombine[(dfcombine['Charge ID'].str[0].str.isdigit()) | (dfcombine['Charge ID'].str.startswith("I-"))]

#Transactions starting with tr_ -----> Apple & Google
dfag = dfcombine[dfcombine['Charge ID'].str.contains("tr_") ]
#Others
dfmerge=pd.concat([dfs,dfag,dfpp])
dfother = dfcombine[~dfcombine["Charge ID"].isin(dfmerge["Charge ID"])]

writer = pd.ExcelWriter(outpath + filename, engine='xlsxwriter')
dfs.to_excel(writer,sheet_name="Stripe", index=False)
dfpp.to_excel(writer,sheet_name="Paypal", index=False)
dfag.to_excel(writer,sheet_name="Apple&Google", index=False)
dfother.to_excel(writer,sheet_name="Others", index=False)

writer.save()
writer.close()

print("Done!")

net_cash_flow-vidiq-2023-02-Year.xlsx
net_cash_flow-vidiq-2023-02-Mon.xlsx
Done!


/usr/local/lib/python3.8/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
